In [80]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from pygoogletranslation import Translator

<h1> Data Analysis Mini-project using Pandas & Matplotlib</h1>
<h3>Sudent: Adam Sypnik </h2>
<h3>Album: 136151 </h3>

<h4>Ex1. Read the data and view it.<h4>
<p><b>Important:</b> add the parameter "encoding='latin1', to.pd_read_csv() because the data is in Portuguese.
(‘á’ and ‘ç’ will generate an error).</p>

In [90]:
df = pd.read_csv('Data/amazon.csv', sep=',',encoding='latin1' , na_filter=True,)
df.head(10)

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01
5,2003,Acre,Janeiro,10.0,2003-01-01
6,2004,Acre,Janeiro,0.0,2004-01-01
7,2005,Acre,Janeiro,12.0,2005-01-01
8,2006,Acre,Janeiro,4.0,2006-01-01
9,2007,Acre,Janeiro,0.0,2007-01-01


<h4>Ex2. The months in column "month" are in Portuguese. Change them to English</h4>

In [32]:
tr = Translator()
df['month'] = df['month'].apply(lambda x: tr.translate(x,dest='en').text)

Exception: Unexpected status code 429 from https://translate.google.com/_/TranslateWebserverUi/data/batchexecute after retried 3 loop with 5s delay

<h4>Ex3. Missing values in state column:
<p>
<ol type="a">
    <li> - randobly delete 200 values from the state column</li>
    <li> - Check for the missing values in this column </li>
    <li> - Fil each missing value with that of the previous (or next) row. </li>
</ol>

<p> Randomly delete 200 values from the state column</p>

In [91]:
i = 0
t = []
while i<200:
    temp = np.random.randint(len(df['state']))
    # print('First temp {}'.format(temp))
    if not pd.isna(df['state'].loc[temp]):
        df['state'].loc[temp] = np.NaN
        i+=1

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


<p> Check for missing values in this column</p>

In [96]:
df.isnull().sum()

year        0
state     200
month       0
number      0
date        0
dtype: int64

In [98]:
temp1 = df[df['state'].isnull() == True].index.tolist()
for i in temp1:
    df['state'].loc[i] = df['state'].loc[i-1]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [99]:
df['state'].isnull().sum()

0

In [ ]:
df_temp= df
# df_temp.isnull().sum()
df_temp.isnull().sum()
